In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import pylab

import pandas as pd
import xgboost as xgb
import statsmodels.stats.api as sm

from sklearn import ensemble , cross_validation, learning_curve, metrics 
from sklearn.cross_validation import train_test_split

from sklearn import preprocessing

from sklearn.metrics import mean_squared_error
import math

from sklearn.ensemble import RandomForestClassifier
from numpy import genfromtxt, savetxt
import matplotlib.pyplot as plt

%matplotlib inline

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [3]:
FULLDATA = pd.read_csv("data_recruiting_bi_data_challenge.csv", sep=',', header=0)


In [4]:
FULLDATA.tail()

,db_id,hotel_id,city_id,clicks,stars,distance_to_center,avg_price_hotel,rating,nmbr_partners_index,avg_rel_saving,avg_rank
832080,832081,1.090017e+11,25084,NaN,0,104939,NaN,NaN,NaN,NaN,NaN
832081,832082,1.090029e+11,23060,NaN,0,33386,NaN,NaN,NaN,NaN,NaN
832082,832083,1.090038e+11,184186,0.0,2,59788,NaN,NaN,NaN,NaN,NaN
832083,832084,1.090042e+11,69496,0.0,0,72578,NaN,NaN,NaN,NaN,NaN
832084,832085,1.090045e+11,99157,0.0,0,116588,NaN,NaN,NaN,NaN,NaN


In [5]:
clean_index = [np.isnan(FULLDATA).sum(axis=1) < 1 ]

In [39]:
np.sum(clean_index[0])*100/np.size(clean_index[0], axis=0)

13.812891711784253

In [11]:
type(clean_index)

list

In [21]:
np.shape(clean_index)

(1, 832085)

In [38]:
np.size(FULLDATA, axis=0)

832085

In [6]:
np.size(FULLDATA[clean_index[0]], axis=0)/np.size(FULLDATA, axis=0)

0.13812891711784253

In [7]:
DATASET_clean = FULLDATA[clean_index[0]].copy()

In [8]:
DATASET_clean.tail()

,db_id,hotel_id,city_id,clicks,stars,distance_to_center,avg_price_hotel,rating,nmbr_partners_index,avg_rel_saving,avg_rank
814938,814939,1.005929e+11,61370,69.0,4,1915,179.0,88.0,1.72,17.0,9.7541
815992,815993,1.007070e+11,84207,0.0,0,12957,113.0,73.0,0.48,0.0,86.0000
817860,817861,1.009202e+11,128276,1.0,2,521,54.0,75.0,0.16,0.0,66.7553
818400,818401,1.009850e+11,81393,0.0,2,7410,54.0,78.0,0.19,0.0,932.0000
818860,818861,1.010841e+11,81393,1.0,0,9479,42.0,74.0,0.30,14.0,356.0000


In [9]:
X_train_v1, X_test_v1, Y_train_v1, Y_test_v1 = train_test_split(DATASET_clean.iloc[:, DATASET_clean.columns != 'clicks'], DATASET_clean['clicks'], test_size=0.25, random_state=42)

In [10]:
n_trees = list(range(1, 100, 50) )

In [11]:
X_train_v1_fit = X_train_v1.drop(["db_id", "hotel_id"], axis=1)
X_test_v1_fit = X_test_v1.drop(["db_id", "hotel_id"], axis=1)

In [12]:
X_train_v1_fit.head()

,city_id,stars,distance_to_center,avg_price_hotel,rating,nmbr_partners_index,avg_rel_saving,avg_rank
106854,33701,4,1063,79.0,83.0,1.19,11.0,2.8491
731055,31499,2,7463,33.0,79.0,0.23,0.0,759.0000
171603,20375,0,414,74.0,77.0,0.65,0.0,7.9060
99381,61949,0,2401,38.0,76.0,0.20,1.0,332.2544
45258,14704,1,6720,89.0,72.0,1.38,8.0,14.3651


In [60]:
%%time
scoring = []
for n_tree in n_trees:
    estimator = ensemble.RandomForestClassifier(n_estimators = n_tree, min_samples_split=5, random_state=42)
    score = cross_validation.cross_val_score(estimator, X_train_v1_fit, Y_train_v1, 
                                             scoring = 'accuracy', cv = 5)    
    scoring.append(score)
scoring = np.asmatrix(scoring)

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)
/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


KeyboardInterrupt: 

In [13]:
estimator = ensemble.RandomForestClassifier(n_estimators = 50, min_samples_split=5, random_state=42)

In [ ]:
estimator.fit(X_train_v1_fit, Y_train_v1)

In [61]:
importances = estimator.feature_importances_
std = np.std([tree.feature_importances_ for tree in estimator.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train_fit.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, X_train_fit.columns[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train_fit.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X_train_fit.shape[1]), X_train_fit.columns[indices])
plt.xlim([-1, X_train_fit.shape[1]])
plt.show()

NotFittedError: Estimator not fitted, call `fit` before `feature_importances_`.